
# CAE-LSTM降阶模型——Sod激波管

## 概述

为有效降低使用CFD方法的设计成本和周期，近年来降阶模型得到了广泛的关注。对于复杂的可压缩流动，使用本征正交分解(POD)等线性方法进行流场降维，需要大量的模态才能保证流场重建的精度。已有研究表明，采用非线性降维方法能够有效减少所需模态数。卷积自编码器(CAE)是一种由编码器和解码器组成的神经网络，能够实现数据降维和重构，可看作是POD方法的非线性拓展。采用CAE进行流场数据的非线性降维，同时使用长短期记忆神经网络(LSTM)进行流场状态的时间演化。针对非定常可压缩流动，CAE-LSTM降阶模型能够在使用较少自由变量数的前提下获得较高的重构和预测精度。

## 模型架构

CAE-LSTM降阶模型采用CAE网络对流场进行降维，提取流动数据的特征，将其压缩到编码器的隐藏空间中，再用LSTM网络对隐藏空间的自由变量进行系数时间演化，得到流动其他时刻的自由变量，再通过CAE网络的解码器将演化的自由变量进行解码，重建得到相应时刻的流场流动数据。CAE-LSTM流动降阶模型的构造依赖于CAE网络的数据降维和LSTM网络的系数时间演化。与现有的POD/DMD等方法相比，使用CAE网络对流场数据进行非线性降维，同时使用LSTM网络对自由变量进行无方程演化，可以在保证流场降阶模型具备一定精度的情况下，得到更高的压缩比，提高流场预测的效率。

+ 输入：输入一段时间的流场
+ 压缩：通过CAE的编码器对流场进行降维，提取高维时空流动特征
+ 演化：通过LSTM学习低维空间流场时空特征的演变，预测下一时刻
+ 重建：通过CAE的解码器将预测的流场低维特征恢复到高维空间
+ 输出：输出对下一时刻瞬态流场的预测结果

训练时，首先进行CAE网络的训练，训练完成之后使用CAE的编码器得到流场的低维特征，将此低维特征作为LSTM网络的数据集，进行LSTM网络的训练。

![CAE-LSTM.png](./images/CAE-LSTM.png)

## CAE网络数据集

一维Sod激波管

+ Sod激波管问题是一维无黏可压缩流动的初始间断演化问题，数据为数值仿真流场。Sod激波管坐标x范围为\[0, 1\]，中间x=0.5处有一薄膜。在初始时刻，将激波管中间的薄膜撤去，研究激波管中气体密度的变化情况。计算时间t范围为\[0, 0.2\]，平均分成531个时间步。 共531张流场快照，每张快照矩阵尺寸为256

In [1]:
import os
import time
import argparse
import numpy as np

from mindspore import nn, ops, context, save_checkpoint, set_seed, jit, data_sink
from mindflow.utils import load_yaml_config

In [2]:
from src import create_cae_dataset, CaeNet, plot_train_loss

## 训练环境

+ 训练采用Mindspore框架的静态图模式（GRAPH）
+ 在GPU（默认）或Ascend进行训练（单卡）

In [3]:
np.random.seed(0)
set_seed(0)

In [4]:
parser = argparse.ArgumentParser(description='cae net for sod')
parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Context mode, support 'GRAPH', 'PYNATIVE'")
parser.add_argument("--save_graphs", type=bool, default=False, choices=[True, False],
                    help="Whether to save intermediate compilation graphs")
parser.add_argument("--save_graphs_path", type=str, default="./graphs")
parser.add_argument("--device_target", type=str, default="GPU", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'Ascend', 'GPU'")
parser.add_argument("--device_id", type=int, default=0, help="ID of the target device")
parser.add_argument("--config_file_path", type=str, default="./config.yaml")
args = parser.parse_args()

context.set_context(mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs,
                    save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target,
                    device_id=args.device_id)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

## CAE网络超参数

In [5]:
config = load_yaml_config(args.config_file_path)
data_params = config["cae_data"]
model_params = config["cae_model"]
optimizer_params = config["cae_optimizer"]

## 训练过程文件保存路径

In [6]:
summary_dir = optimizer_params["summary_dir"]
if not os.path.exists(summary_dir):
    os.mkdir(summary_dir)
ckpt_dir = os.path.join(summary_dir, 'ckpt')
if not os.path.exists(ckpt_dir):
    os.mkdir(ckpt_dir)

## 构建CAE网络

In [7]:
cae = CaeNet(model_params["data_dimension"], model_params["conv_kernel_size"], model_params["maxpool_kernel_size"],
             model_params["maxpool_stride"], model_params["encoder_channels"], model_params["decoder_channels"])

## CAE网络优化器

In [8]:
loss_fn = nn.MSELoss()
cae_opt = nn.Adam(cae.trainable_params(), optimizer_params["lr"], weight_decay=optimizer_params["weight_decay"])

if use_ascend:
    from mindspore.amp import DynamicLossScaler, auto_mixed_precision, all_finite
    loss_scaler = DynamicLossScaler(1024, 2, 100)
    auto_mixed_precision(cae, 'O1')
else:
    loss_scaler = None

## CAE网络训练框架

In [9]:
def forward_fn(data, label):
    logits = cae(data)
    loss = loss_fn(logits, label)
    if use_ascend:
        loss = loss_scaler.scale(loss)
    return loss

grad_fn = ops.value_and_grad(forward_fn, None, cae_opt.parameters, has_aux=False)

@jit
def train_step(data, label):
    loss, grads = grad_fn(data, label)
    if use_ascend:
        loss = loss_scaler.unscale(loss)
        if all_finite(grads):
            grads = loss_scaler.unscale(grads)
    loss = ops.depend(loss, cae_opt(grads))
    return loss

## CAE网络数据集加载

In [10]:
cae_dataset, _ = create_cae_dataset(data_params["data_path"], data_params["batch_size"])

## 数据下沉设置

In [11]:
sink_process = data_sink(train_step, cae_dataset, sink_size=1)
train_data_size = cae_dataset.get_dataset_size()

# CAE网络模型训练

In [12]:
print(f"====================Start cae train=======================")
train_loss = []
for epoch in range(1, optimizer_params["epochs"] + 1):
    local_time_beg = time.time()
    cae.set_train()
    epoch_train_loss = 0
    for _ in range(train_data_size):
        epoch_train_loss = ops.squeeze(sink_process(), axis=())
    train_loss.append(epoch_train_loss)
    print(f"epoch: {epoch} train loss: {epoch_train_loss} epoch time: {time.time() - local_time_beg:.2f}s")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(cae, f"{ckpt_dir}/cae_{epoch}.ckpt")
print(f"=====================End cae train========================")
plot_train_loss(train_loss, summary_dir, optimizer_params["epochs"], "cae")

pid:23104
====================Start cae train=======================
epoch: 1 train loss: 0.00859989 epoch time: 3.23s
epoch: 2 train loss: 0.00563688 epoch time: 0.52s
epoch: 3 train loss: 0.00485115 epoch time: 0.53s
epoch: 4 train loss: 0.00341164 epoch time: 0.62s
epoch: 5 train loss: 0.00332990 epoch time: 0.57s
...
epoch: 4396 train loss: 3.69731242e-06 epoch time: 0.51s
epoch: 4397 train loss: 2.65247831e-06 epoch time: 0.55s
epoch: 4398 train loss: 1.14417275e-06 epoch time: 0.54s
epoch: 4399 train loss: 4.97764995e-06 epoch time:0.52s
epoch: 4400 train loss: 2.48092419e-06 epoch time: 0.55s
====================End cae train=======================


## CAE流场重建结果

+ 在训练完CAE网络后，可运行cae_prediction.py查看CAE的训练结果，以判断是否继续进行LSTM网络的训练
+ 下图分别为真实流场，CAE流场重建结果以及平均相对误差

<figure class="harf">
    <img src="./images/true.gif" title="cae_train_loss" width="300"/>
    <img src="./images/cae.gif" title="cae_prediction" width="300"/>
    <img src="./images/cae_error.png" title="cae_error" width="300"/>
</center>

## LSTM网络

LSTM网络框架搭建、训练等相关处理与CAE网络类似。

## 训练环境

In [13]:
import os
import time
import argparse
import numpy as np

from mindspore import nn, ops, context, save_checkpoint, set_seed, jit, data_sink
from mindflow.utils import load_yaml_config

from src import create_lstm_dataset, Lstm, plot_train_loss
from cae_prediction import cae_prediction

np.random.seed(0)
set_seed(0)

parser = argparse.ArgumentParser(description='lstm net for sod')
parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Context mode, support 'GRAPH', 'PYNATIVE'")
parser.add_argument("--save_graphs", type=bool, default=False, choices=[True, False],
                    help="Whether to save intermediate compilation graphs")
parser.add_argument("--save_graphs_path", type=str, default="./graphs")
parser.add_argument("--device_target", type=str, default="GPU", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'Ascend', 'GPU'")
parser.add_argument("--device_id", type=int, default=0, help="ID of the target device")
parser.add_argument("--config_file_path", type=str, default="./config.yaml")
args = parser.parse_args()

context.set_context(mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs,
                    save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target,
                    device_id=args.device_id)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

## LSTM网络框架及训练设置

In [14]:
# prepare params
config = load_yaml_config(args.config_file_path)
data_params = config["lstm_data"]
model_params = config["lstm_model"]
optimizer_params = config["lstm_optimizer"]

# prepare summary file
summary_dir = optimizer_params["summary_dir"]
ckpt_dir = os.path.join(summary_dir, 'ckpt')

# prepare model
lstm = Lstm(model_params["latent_size"], model_params["hidden_size"], model_params["num_layers"])

loss_fn = nn.MSELoss()
lstm_opt = nn.Adam(lstm.trainable_params(), optimizer_params["lr"], weight_decay=optimizer_params["weight_decay"])
if use_ascend:
    from mindspore.amp import DynamicLossScaler, auto_mixed_precision, all_finite
    loss_scaler = DynamicLossScaler(1024, 2, 100)
    auto_mixed_precision(lstm, 'O1')
else:
    loss_scaler = None

# Define forward function
def forward_fn(data, label):
    logits = lstm(data)
    loss = loss_fn(logits, label)
    if use_ascend:
        loss = loss_scaler.scale(loss)
    return loss

# Get gradient function
grad_fn = ops.value_and_grad(forward_fn, None, lstm_opt.parameters, has_aux=False)

@jit
def train_step(data, label):
    loss, grads = grad_fn(data, label)
    if use_ascend:
        loss = loss_scaler.unscale(loss)
        if all_finite(grads):
            grads = loss_scaler.unscale(grads)
    loss = ops.depend(loss, lstm_opt(grads))
    return loss

## LSTM网络数据集加载与处理

LSTM网络数据集由CAE的编码器得到

In [15]:
# prepare dataset
latent_true = cae_prediction()
lstm_dataset, _ = create_lstm_dataset(latent_true, data_params["batch_size"], data_params["time_size"],
                                      data_params["latent_size"], data_params["time_window"],
                                      data_params["gaussian_filter_sigma"])

# data sink
sink_process = data_sink(train_step, lstm_dataset, sink_size=1)
train_data_size = lstm_dataset.get_dataset_size()

## LSTM网络模型训练

In [16]:
print(f"====================Start lstm train=======================")
train_loss = []
for epoch in range(1, optimizer_params["epochs"] + 1):
    local_time_beg = time.time()
    lstm.set_train()
    epoch_train_loss = 0
    for _ in range(train_data_size):
        epoch_train_loss = ops.squeeze(sink_process(), axis=())
    train_loss.append(epoch_train_loss)
    print(f"epoch: {epoch} train loss: {epoch_train_loss} epoch time: {time.time() - local_time_beg:.2f}s")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(lstm, f"{ckpt_dir}/lstm_{epoch}.ckpt")
print(f"=====================End lstm train========================")
plot_train_loss(train_loss, summary_dir, optimizer_params["epochs"], "lstm")

====================Start lstm train=======================
epoch: 1 train loss: 0.4425844 epoch time: 3.75s
epoch: 2 train loss: 0.23611887 epoch time: 0.75s
epoch: 3 train loss: 0.65945524 epoch time: 0.76s
epoch: 4 train loss: 0.77271056 epoch time: 0.80s
epoch: 5 train loss: 0.3535387 epoch time: 0.81s
...
epoch: 4396 train loss: 9.665465e-05 epoch time: 0.76s
epoch: 4397 train loss: 5.5045904e-05 epoch time: 0.77s
epoch: 4398 train loss: 0.00013155791 epoch time: 0.80s
epoch: 4399 train loss: 0.0769522e-05 epoch time: 0.80s
epoch: 4400 train loss: 0.0010389996 epoch time: 0.80s
====================End lstm train=======================


## 预测流场结果可视化

+ 运行cae_lstm_prediction.py可查看CAE-LSTM降阶模型的预测结果
+ 下图分别为真实流场，CAE-LSTM网络的预测结果和相对应的平均相对误差

<figure class="harf">
    <img src="./images/true2.gif" title="cae_prediction" width="300"/>
    <img src="./images/cae_lstm.gif" title="cae_lstm_prediction" width="300"/>
    <img src="./images/cae_lstm_error.png" title="cae_lstm_error" width="300"/>
</center>